In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import time
from functions import basic
from functions import interaction

In [2]:
%%javascript
require(["codemirror/keymap/sublime", "notebook/js/cell", "base/js/namespace"],
    function(sublime_keymap, cell, IPython){
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
        var cells = IPython.notebook.get_cells();
        for(var cl=0; cl< cells.length ; cl++){
            cells[cl].code_mirror.setOption('keyMap', 'sublime');
        }
    }
);

<IPython.core.display.Javascript object>

In [22]:
filename = "fcc100a256.txt"
mask,distances = basic.find_neighbours(*basic.read_file(filename))

In [23]:
start = time.process_time()
LJA = interaction.lennard_jones_approx(distances)
print(time.process_time() - start)

start = time.process_time()
LJ = interaction.lennard_jones(distances)
print(time.process_time() - start)

LJA,LJ

0.02633887300000004
0.013228440999999869


(-453.45018064937494, -532.480009525025)

### Test zone - used for quick tests and monkey patch

In [5]:
import time
start = time.process_time()
# your code here    
print(time.process_time() - start)

4.896599999992812e-05


In [6]:
# constant values 
rc = 4.625
rp = 4.375
sigma = 2.644
epsilon = 0.345
kb = 1/11603
Temp = 100
m_ag = 108*1.66e-27/16

In [31]:
def lennard_jones(distances):
    '''
    returns total potential energy value
    '''
    distances[np.where(distances==0)] = np.infty
    Epot = 4*epsilon*((sigma/(distances))**12 - (sigma/(distances))**6)
    return 0.5*np.nansum(Epot)

In [32]:
LJ = lennard_jones(distance)
LJ

-532.480009525025

In [28]:
distances[np.where(distances==0)] = np.infty

In [29]:
np.shape(distances)

(256, 256)

In [30]:
distances

array([[        inf,  2.94184705,  2.94184705, ..., 18.37182895,
        21.81732166, 21.81732166],
       [ 2.94184705,         inf,  4.1604    , ..., 20.16828728,
        24.08005593, 24.08005593],
       [ 2.94184705,  4.1604    ,         inf, ..., 19.29098119,
        22.21045818, 21.81732166],
       ...,
       [18.37182895, 20.16828728, 19.29098119, ...,         inf,
         6.57816999,  8.3208    ],
       [21.81732166, 24.08005593, 22.21045818, ...,  6.57816999,
                inf,  2.94184705],
       [21.81732166, 24.08005593, 21.81732166, ...,  8.3208    ,
         2.94184705,         inf]])